# Generate the figures for soil properties presented in the paper

In [1]:
import pandas as pd
import matplotlib as mpl
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import seaborn as sns
# plt.style.use('seaborn-deep')
# plt.style.use("seaborn-v0_8")
# sns.set_theme()
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
%matplotlib inline

In [2]:
sns = None
mpl.rc_file_defaults()

In [4]:
def evaluate(measured, predicted):
    r2score = r2_score(measured, predicted)
    print('R2 Score:', r2score)
    
    # calculate IQR
    q1 = np.percentile(predicted, 25)
    q3 = np.percentile(predicted, 75)
    iqrCal = q3 - q1

    # calculate MAE
    mae = mean_absolute_error(measured, predicted)

    # calculate RPIQ
    rpiq = mae / iqrCal
    print("RPIQ:", rpiq)
    return r2score, rpiq

def curveFitting(measured, predicted, degree):
    poly = np.polyfit(measured, predicted, degree)
    if degree == 1:
        x_range = []
#         x_range = np.linspace(np.min(measured), np.max(measured) + 0.05, 3)
        poly_trend = np.poly1d(poly)#(x_range)
    elif degree > 1:
        x_range = np.linspace(np.min(measured), np.max(measured) + 0.5, 100)
        poly_trend = np.poly1d(poly)(x_range)
    y_hat = np.poly1d(poly)(measured)
    
#     rsquared = r2_score(measured, predicted)
    
    rsquared = 1 - (((measured - predicted) ** 2).sum() / ((measured - np.mean(predicted)) ** 2).sum())
    
    # calculate IQR
    q1 = np.percentile(predicted, 25)
    q3 = np.percentile(predicted, 75)
    iqrCal = q3 - q1

    # calculate MAE
    mae = mean_absolute_error(measured, predicted)

    # calculate RPIQ
    rpiq = mae / iqrCal
    
    # calculate MSE
    mse = mean_squared_error(measured, predicted)
    rmse = np.sqrt(mse)
    
    return poly, x_range, poly_trend, rsquared, rpiq, rmse

def poly2latex(poly, variable = "x", width = 3):
    t = ["{0:0.{width}f}"]
    t.append("(" + t[-1] + ") " + " {variable}")
    t.append(t[-1] + "^{1}")
    def f():
        for i, v in enumerate(reversed(poly)):
            idx = i if i < 2 else 2
            yield t[idx].format(v, i, variable=variable, width=width)
    return "${}$".format("+".join(f()))

def plotSlope(measured, predicted, soilPro, sheet_name, npts, val_limit):
    %matplotlib inline
    edge = 'black'
    point_size = 60
    fig, ax = plt.subplots(1,1, figsize = (4.2, 4.2))

    X = measured
    y = predicted
    
#     minX = math.floor(X.min())
#     maxX = math.ceil(X.max())
#     stdX = np.floor(np.std(X))
    
    poly, x_range, poly_trend, rsquared, rpiq, rmseB = curveFitting(X, y, 1)
    ax.scatter(X, y, c = ['red'], edgecolors = edge, s = point_size)
    ax.set_xlabel('Measured ' + soilPro, fontsize=12)
    ax.set_ylabel('Predicted ' + soilPro, fontsize=12)
    ax.set_ylim(val_limit)
    ax.set_xlim(val_limit)
    
    ax.plot([min(X), max(X)], poly_trend([min(X), max(X)]), '--', color='blue', linewidth=1.5)
#     ax.plot(X, poly_trend(X), '--', color='blue', linewidth=1.5)
#     plt.gca().text(0.05, 0.95, poly2latex(poly), fontsize=10, verticalalignment='top',horizontalalignment='left', transform=ax.transAxes)
    textRSq = f"\n$Npts = {npts}$\n$R^2 = {rsquared:0.3f}$\n$RMSE = {rmseB:0.3f}$\n$RPIQ = {rpiq:0.3f}$"
    plt.gca().text(0.05, 0.99, textRSq, fontsize=10, verticalalignment='top',horizontalalignment='left', transform=ax.transAxes)
    plt.tight_layout()
    plt.savefig('./5_Predicted_SoilProperties/'+ sheet_name + '_' + soilPro \
                + '.png', dpi=300, bbox_inches='tight')
    time.sleep(2)
    plt.close()

### Soil Properties
'Organic Matter (%)'

'CaCO3 (%)'

'Higroscopic Water (%)'

'Fe dithionite (ppm)'

'S.S.A (m2/g)'

'Al2O3'

In [5]:
soilPro = 'Organic Matter (%)'
fname = './4_Calibration_Plots/Organic Matter_Output_Predicted.xlsx'
df = pd.ExcelFile(fname)
print(df.sheet_names)
data = pd.read_excel(fname, sheet_name='Cal_PLSR')
actual = data[soilPro]
pred = data["Predicted"]
minX = math.floor(actual.min())
maxX = math.ceil(actual.max())
stdX = np.floor(np.std(actual))
minY = math.floor(pred.min())
maxY = math.ceil(pred.max())
stdY = np.floor(np.std(pred))
print('Xlimits =', [minX, maxX, stdX])
print('Ylimits =', [minY, maxY, stdY])

['Cal_PLSR', 'Test_PLSR', 'Cal_RR', 'Test_RR', 'Cal_LR', 'Test_LR']
Xlimits = [0, 7, 1.0]
Ylimits = [0, 6, 1.0]


In [6]:
for sheets in df.sheet_names:
    sheet_name = sheets
    val_limit = [0, 8]
    dfData = pd.read_excel(fname, sheet_name=sheets)
    dfData.dropna(axis=0, inplace=True)
    measured = dfData[soilPro]
    predicted = dfData["Predicted"]
    npts = len(measured)
    plotSlope(measured, predicted, soilPro, sheet_name, npts, val_limit)